In [1]:
# Import necessary libraries
import asyncio
import sys
import os
import warnings
from devtools import pprint
# Add the src directory to the Python path and suppress warnings
sys.path.append(os.path.abspath('../src/agent'))
warnings.filterwarnings("ignore", category=FutureWarning)
from market_intelligence_agent.config import TEAM_MEMBERS
from langchain_core.messages import convert_to_messages
from langgraph_sdk import get_client
from datetime import datetime
import json
import logging
import uuid


In [2]:
# LangGraph Client
lg_client = get_client(url="http://localhost:2024")

user_query = ["what is outlook of Mag 7 for 2025. Include the technicals set up and the events/sentiments set up. Pitch me a trade idea at the end. Do not use coder"]
user_input_messages = user_query

thread = await lg_client.threads.create()

print("Starting graph stream...")
# NOTE: The input dictionary is simplified
async for chunk in lg_client.runs.stream(
    thread["thread_id"],
    assistant_id="market_intelligence_agent",
    input={
        # Constants
        "TEAM_MEMBERS": TEAM_MEMBERS,
        # Runtime Variables
        "messages": user_input_messages,
        "current_timestamp": datetime.now(),
        "researcher_credits": 6,
        "market_credits": 6,
        "coder_credits": 0,
        "browser_credits": 3,
    },
    config={
        "recursion_limit": 150
    },
    stream_mode="values"
):  
    messages = chunk.data.get("messages", None)
    state = chunk.data
    next_agent = chunk.data.get('next', None)
    last_agent = chunk.data.get('last_agent', None)
    print(chunk.data)

    if messages:
        print("-" * 120)
        agent_name = messages[-1].get('name', None)
        if agent_name:
            if agent_name == "planner":
                full_plan = state.get('full_plan', None)
                print(json.loads(convert_to_messages(messages)[-1].content)['thought'])
                print(f"Planner handed off the following plan to the supervisor: \n{full_plan}")
            if agent_name == "supervisor":
                instructions = json.loads(convert_to_messages(messages)[-1].content)
                print(f"Supervisor assigned the following task to {next_agent}: \n{instructions['task']}")
            if agent_name == "researcher":
                research_result = json.loads(convert_to_messages(messages)[-1].content)
                print(f"Researcher has finished the task: {research_result['result_summary']}")
            if agent_name == "coder":
                coder_result = json.loads(convert_to_messages(messages)[-1].content)
                print(f"Coder has finished the task: {coder_result['result_summary']}")
            if agent_name == "market":
                market_result = json.loads(convert_to_messages(messages)[-1].content)
                print(f"Market agent has finished the task: {market_result['result_summary']}")
    if next_agent:
        print("=" * 120)
        if next_agent == "planner":
            print('Planner is thinking...')
        if next_agent == "supervisor":
            print(f'Supervisor is evaluating response from {last_agent}...')
        if next_agent == "researcher":
            print('Researcher is gathering information...')
        if next_agent == "coder":
            print('Coder is coding...')
        if next_agent == "market":
            print('Market agent is retrieving market data...')
        if next_agent == "browser":
            print('Browser agent is browsing the web...')
        if next_agent == "analyst":
            print('Analyst agent is analyzing the gathered information...')
        if next_agent == "reporter":
            print('Reporter agent is preparing the final report...')
        if last_agent == "coordinator":
            print('Coordinator handed off the task to the team.')

print("Graph stream finished.")



Starting graph stream...
{'run_id': '1f021da2-e3bf-6e0e-b6de-9afe5fee6c95', 'attempt': 1}
{'messages': [{'content': 'what is outlook of Mag 7 for 2025. Include the technicals set up and the events/sentiments set up. Pitch me a trade idea at the end. Do not use coder', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'human', 'name': None, 'id': 'efd3644f-d3cd-4ffb-ab5a-dac6d78f8fee', 'example': False}], 'TEAM_MEMBERS': ['researcher', 'coder', 'reporter', 'market', 'browser', 'analyst'], 'current_timestamp': '2025-04-25T09:35:41.073067', 'researcher_credits': 6, 'coder_credits': 0, 'browser_credits': 3, 'market_credits': 6}
------------------------------------------------------------------------------------------------------------------------
{'messages': [{'content': 'what is outlook of Mag 7 for 2025. Include the technicals set up and the events/sentiments set up. Pitch me a trade idea at the end. Do not use coder', 'additional_kwargs': {}, 'response_metadata': {}, 'type': 'h

CancelledError: 

In [3]:
current_state = await lg_client.threads.get_state(thread["thread_id"])

In [4]:
current_state

{'values': {'messages': [{'content': 'what is outlook of Mag 7 for 2025. Include the technicals set up and the events/sentiments set up. Pitch me a trade idea at the end. Do not use coder',
    'additional_kwargs': {},
    'response_metadata': {},
    'type': 'human',
    'name': None,
    'id': '069a454f-8e0e-4a40-ab99-ca217e16c895',
    'example': False},
   {'content': '{\n  "thought": "The user wants a comprehensive outlook for the \'Mag 7\' stocks for 2025, including both technical analysis and key event/sentiment drivers. The Mag 7 usually refers to seven top tech-related stocks driving market movements, often including Apple, Microsoft, Amazon, Alphabet, Meta, Nvidia, and Tesla, but this needs to be verified. A valuable response should cover: identifying the current and projected technical setup for each stock, key upcoming events or sentiment factors impacting them in 2025, and how macroeconomic and sector trends could influence their trajectories. The final deliverable should 

In [5]:
from IPython.display import Markdown, display

def display_markdown(content):
    start_marker = "<response>\n"
    end_marker = "</response>\n"
    start_index = content.find(start_marker)
    end_index = content.find(end_marker)
    if start_index != -1 and end_index != -1:
        content = content[start_index + len(start_marker):end_index]
    return Markdown(content)
    
    
# Display the formatted markdown
display(display_markdown(current_state['values']['final_report']))

# Mag 7 Stocks 2025 Outlook: Integrated Technical, Fundamental, and Macro Analysis with Trade Recommendation

---

## Executive Summary  
The Mag 7 — Apple, Microsoft, Alphabet, Amazon, Meta Platforms, Nvidia, and Tesla — remain pivotal drivers of market returns in 2025 amid a complex backdrop of accelerating AI innovation, regulatory scrutiny, US-China tariff tensions, and sector rotation dynamics. Technicals depict mixed consolidation and selective bullish momentum. Fundamentals highlight strong growth potential, especially in AI and cloud-related businesses, but also reveal valuation dispersion and elevated volatility. Regulatory and geopolitical risks add near-term uncertainties, particularly for Apple, Meta, and Amazon. A risk-conscious, diversified portfolio emphasizing AI/cloud leaders with hedging around more volatile names emerges as the optimal strategy for 2025.

---

## Market Context and Macroeconomic Environment  
- **Geopolitical/Trade:** Elevated US-China tariffs and supply chain shifts persist, prompting production diversification (e.g., Apple moving assembly to India). Tariff relief talks are ongoing but unresolved.  
- **Regulatory:** EU Digital Markets Act and antitrust enforcement impose fines and operational constraints on key players (Apple, Meta, Alphabet). Heightened privacy and competition scrutiny weigh on sentiment.  
- **Sector Rotation:** Defensive sectors outperform amid economic caution; within technology, semiconductors and AI infrastructure retain investor favor, supporting Nvidia and Microsoft’s cloud services.  
- **Volatility:** Elevated market volatility due to mixed earnings data and uncertain macro landscape. Investors show caution balancing growth ambition with risk containment.

---

## Mag 7 Individual Stock Outlook

| Stock     | Technical Setup                                      | Fundamental Snapshot                       | Key Event/Sentiment Drivers                          | Main Risks                                 |
|-----------|-----------------------------------------------------|--------------------------------------------|----------------------------------------------------|-------------------------------------------|
| **Apple** | Consolidating $175–190 range; neutral momentum.     | P/E 33.1; premium vs DCF; 10% growth.      | Production shift to India; product refresh cycles; EU fines. | Supply chain complexity; regulatory costs. |
| **Microsoft** | Bullish trend; support near $315; strong MA combo. | P/E 31.3; growth 14%; DCF slightly below current price. | Cloud AI expansion; collaboration deals; tariff insulation due to services model. | Competitive saturation.                   |
| **Alphabet** | Stable base ~$120–130; resistance near $135–140.  | P/E 19.8; undervalued vs DCF; growth 8.5%. | AI/ Gemini rollout; antitrust litigation ongoing. | Regulatory fines; geopolitical tensions.   |
| **Amazon** | Bearish but oversold near $115–120 support.         | P/E 33.8; growth 20%; undervalued vs DCF.   | AWS growth; retail competition; tariff cost pressures. | Tariff inflation; slowing retail segment.  |
| **Meta**  | Volatile, bottoming $170–180; downward bias.        | P/E 22.3; undervalued vs DCF; growth 14%.  | EU fines; AI pivot; privacy crackdown.             | Regulatory penalties; user stagnation.    |
| **Nvidia**| Strong breakout beyond $100; bullish momentum.      | P/E 36.2; fair valuation; high 28.5% growth.| AI chip demand surge; supply chain easing; export controls. | Export restrictions; competition.         |
| **Tesla** | Bearish trend; support ~$200; high volatility.      | P/E 149; declining earnings; elevated volatility. | CEO political distractions; robotaxi uncertainty; emerging EV competition. | Regulatory/environmental risk; brand impact.|

---

## Integrated Investment Thesis  
The Mag 7’s 2025 trajectory hinges on the balance between secular growth from AI and cloud innovation and near-term risks from regulatory, tariff, and competitive headwinds. Nvidia and Microsoft stand out with strong technical momentum, robust fundamentals, and AI/cloud tailwinds, offering the best risk-adjusted growth exposure. Alphabet represents a stable core holding with attractive valuation and steady cash flow. Apple offers resilience and product cycle catalysts but at a premium price amid regulatory overhangs. Amazon and Meta present compelling medium-term growth growth, yet remain exposed to tariffs and fines, warranting cautious allocation. Tesla embodies the highest risk/reward with fundamental pressures and execution uncertainties but potential upside from autonomous driving innovations.

---

## Trade Recommendation: Diversified, Risk-Aware Positioning

- **Core Overweights:**  
  - **Nvidia (NVDA):** Buy on dips near $90-$110 to capture AI-driven growth and sector leadership.  
  - **Microsoft (MSFT):** Accumulate around $315-$320, leveraging cloud dominance and technical support.  

- **Core Stable Positions:**  
  - **Alphabet (GOOGL):** Build or maintain positions near $120-$130, given undervaluation and regulatory clarity improvements.  
  - **Apple (AAPL):** Scale in gradually on consolidation confirming above $180, balancing premium valuation with product resilience.  

- **Selective Exposure with Risk Management:**  
  - **Amazon (AMZN) & Meta (META):** Tactical, smaller allocations with strict stop-loss protocols; consider hedging downside risk via short-dated put options.  

- **Speculative, Hedged Exposure:**  
  - **Tesla (TSLA):** Maintain limited size focused on volatility trades and mean reversion opportunities. Hedge with protective puts to mitigate downside risk from financial and event uncertainty.

- **Portfolio Hedging:** Deploy options strategies to guard against regulatory shocks and tariff escalations impacting downside risk.

---

## Conclusion  
The Mag 7 stocks in 2025 offer a nuanced mix of high-growth potential anchored in AI and cloud technology, tempered by regulatory and geopolitical risks that elevate volatility and valuation dispersion. A thoughtfully diversified, technically and fundamentally informed portfolio emphasizing AI/cloud leaders (Nvidia, Microsoft), with prudent scaling into stable names (Alphabet, Apple) and guarded stances on more volatile or regulatory-exposed stocks (Amazon, Meta, Tesla), is advisable. Tactical hedging and active monitoring of evolving macro and policy dynamics will be critical to navigating this environment and capturing upside while managing downside.

---

*This outlook integrates empirical fundamentals, technical market data, event-driven risks, and macro considerations to guide portfolio managers toward balanced, evidence-supported positioning within the Mag 7 group for 2025.*

In [7]:
def save_string_to_md(text, directory, filename):
  if not os.path.exists(directory):
    os.makedirs(directory)

  # Create the full file path
  filepath = os.path.join(directory, filename + ".md")

  # Write the string to the file
  with open(filepath, "w") as f:
    f.write(text)

# Example usage:
output_directory = "../assets"
output_filename = "mag7_report_2"

#save_string_to_md(current_state['values']['final_report'], output_directory, output_filename)
